In [5]:
# !pip install datasets transformers scikit-learn

In [3]:
from datasets import load_dataset, DatasetDict, Dataset
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd
import numpy as np

In [4]:
dataset_dict = load_dataset("nickmuchi/financial-classification")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/935 [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/378k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/44.1k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4551 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/506 [00:00<?, ? examples/s]

In [6]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=3)
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [8]:
def tokenize_function(example):
  return tokenizer(example["text"], truncation=True,padding="max_length",max_length=128)
tokenized_datasets = dataset_dict.map(tokenize_function, batched=True)
tokenized_datasets.set_format("torch",columns=["input_ids","attention_mask","labels"])

train_dataset = tokenized_datasets["train"]
test_dataset = tokenized_datasets["test"]

Map:   0%|          | 0/4551 [00:00<?, ? examples/s]

In [10]:
for param in model.distilbert.parameters():
    param.requires_grad = False

In [9]:
def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = logits.argmax(axis=-1)
  return {"accuracy":accuracy_score(labels,predictions)}

In [17]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    eval_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    weight_decay=0.01,
    load_best_model_at_end=True,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_strategy="epoch",
)

In [18]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    compute_metrics = compute_metrics,
)

In [19]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.829000,0.849485,0.571146
2,0.774100,0.797229,0.610672
3,0.737200,0.744002,0.640316
4,0.707500,0.728705,0.650198
5,0.687900,0.692193,0.654150
6,0.668300,0.684402,0.658103
7,0.658300,0.661723,0.679842
8,0.647100,0.648421,0.695652
9,0.634600,0.640772,0.695652
10,0.632500,0.638702,0.683794


TrainOutput(global_step=5700, training_loss=0.6525953406618352, metrics={'train_runtime': 404.3978, 'train_samples_per_second': 225.075, 'train_steps_per_second': 14.095, 'total_flos': 3014349412101120.0, 'train_loss': 0.6525953406618352, 'epoch': 20.0})

In [22]:
import torch

def classify(model, text):
    model.eval()
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128).to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=1)
        pred_class = torch.argmax(probs).item()
        confidence = probs[0][pred_class].item()
    return pred_class, confidence

In [27]:
test_sentence = "Earnings per share EPS rose to EUR 0.11 from EUR 0.03 ."
pred, conf = classify(model, test_sentence)
print(f"Prediction: {pred}")

Prediction: 2
